In [20]:
import torch
import json

In [21]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/MaxMinData.json', 'r') as file:
    data = json.load(file)
PDmax = data['PDmax']
PDmin = data['PDmin']
Tmax = data['Tmax']
Tmin = data['Tmin']
ACCmax = data['ACCmax']
ACCmin = data['ACCmin']

In [22]:
PDmax, PDmin, Tmax, Tmin, ACCmax, ACCmin

(1.9444999999999708, 0.0, 30.5, 25.0, 32.0, 21.0)

In [23]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def ACC_normalize(ACC):
    return ((ACC - ACCmin) / (ACCmax - ACCmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

def ACC_denormalize(ACC):
    return ((ACCmax - ACCmin) * ACC + ACCmin)

In [24]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Transformer_PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Transformer_Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

TransformerModel(
  (embedding): Linear(in_features=10, out_features=16, bias=True)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): TransformerDecod

In [25]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')
ori_data.shape

torch.Size([5716, 30, 10])

In [26]:
ori_data[100][0][0]

tensor(0.1818)

In [27]:
temp_denormalize(ori_data[100][0][0])

tensor(26.)

In [28]:
for i in range(0, 100, 10):
    print('室內溫度 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_denormalize(j)
        # ori_data[i][-3][2] = jj
        # ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        ori_data[i][-1][4] = 0 if(j==32) else 1
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)
        

室內溫度 : 27.14
21 度: 26.0258
22 度: 26.0255
23 度: 26.0251
24 度: 26.0248
25 度: 26.0245
26 度: 26.0243
27 度: 26.024
32 度: 26.0221
室內溫度 : 26.74
21 度: 26.0218
22 度: 26.0216
23 度: 26.0214
24 度: 26.0212
25 度: 26.0211
26 度: 26.0209
27 度: 26.0208
32 度: 26.0193
室內溫度 : 26.36
21 度: 26.0194
22 度: 26.0193
23 度: 26.0192
24 度: 26.0191
25 度: 26.019
26 度: 26.019
27 度: 26.0189
32 度: 26.0177
室內溫度 : 26.2
21 度: 26.0189
22 度: 26.0189
23 度: 26.0188
24 度: 26.0187
25 度: 26.0186
26 度: 26.0186
27 度: 26.0185
32 度: 26.0174
室內溫度 : 26.2
21 度: 26.0175
22 度: 26.0175
23 度: 26.0175
24 度: 26.0175
25 度: 26.0174
26 度: 26.0174
27 度: 26.0174
32 度: 26.0164
室內溫度 : 26.1
21 度: 26.0171
22 度: 26.0171
23 度: 26.0171
24 度: 26.0171
25 度: 26.0171
26 度: 26.0171
27 度: 26.0171
32 度: 26.0161
室內溫度 : 26.0
21 度: 26.0173
22 度: 26.0173
23 度: 26.0173
24 度: 26.0173
25 度: 26.0173
26 度: 26.0173
27 度: 26.0173
32 度: 26.0163
室內溫度 : 26.0
21 度: 26.0161
22 度: 26.0162
23 度: 26.0162
24 度: 26.0162
25 度: 26.0163
26 度: 26.0163
27 度: 26.0163
32 度: 26.0155
室內溫度 : 2